# Debug online endpoints locally in Visual Studio Code

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
)
from azure.identity import DefaultAzureCredential

## Conectar a workspace

In [ ]:
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

## Definir y crear un endpoint local

In [ ]:
local_endpoint_name = "my-local-diabetes-endpoint"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=local_endpoint_name,
    description="Local endpoint for MLflow diabetes model",
    auth_mode="key",
)

In [ ]:
# crear el endpoint
ml_client.begin_create_or_update(endpoint, local=True)

In [ ]:
# obtener el endpoint
ml_client.online_endpoints.get(name=local_endpoint_name, local=True)

## Crear el deployment localmente

In [ ]:
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=local_endpoint_name,
    model=Model(path="./model-1/model/sklearn_regression_model.pkl"),
    code_configuration=CodeConfiguration(
        code="./model-1/onlinescoring", scoring_script="score.py"
    ),
    environment=Environment(
        conda_file="./model-1/environment/conda.yaml",
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    ),
    instance_type="STANDARD_DS1_V2",
    instance_count=1,
)

In [ ]:

deployment = ml_client.online_deployments.begin_create_or_update(
    deployment, local=True, vscode_debug=True
).result()

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint, local=True)

## Obtener detalles de un endpoint

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=local_endpoint_name, local=True)
endpoint

## 2. Debug your endpoint

In [ ]:
endpoint = ml_client.online_endpoints.get(name=local_endpoint_name, local=True)

request_file_path = "./model-1/sample-request.json"

ml_client.online_endpoints.invoke(local_endpoint_name, request_file_path, local=True)

In [ ]:
print(endpoint)

## 3. Edit your endpoint

In [ ]:
new_deployment = ManagedOnlineDeployment(
    name="green",
    endpoint_name=endpoint_name,
    model=Model(path="../model-2/model/sklearn_regression_model.pkl"),
    code_configuration=CodeConfiguration(
        code="../model-2/onlinescoring", scoring_script="score.py"
    ),
    environment=Environment(
        conda_file="../model-2/environment/conda.yaml",
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    ),
    instance_type="Standard_DS3_v2",
    instance_count=2,
)

deployment = ml_client.online_deployments.begin_create_or_update(
    new_deployment, local=True, vscode_debug=True
).result()

## Borrar el endpoint y deployment

In [ ]:
ml_client.online_endpoints.begin_delete(name=local_endpoint_name, local=True)

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=local_endpoint_name, local=True)